In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 50)

In [2]:
reg_data_m =pd.read_csv('data/MRegularSeasonDetailedResults.csv')
reg_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [3]:
ncaa_data_m = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
ncaa_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19


In [4]:
#removing WLoc
print(reg_data_m['WLoc'].value_counts())
print(ncaa_data_m['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    63368
A    33239
N    11027
Name: WLoc, dtype: int64
N    1248
Name: WLoc, dtype: int64


In [5]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_m['NumOT'].value_counts())
print("NCAA\n",ncaa_data_m['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    101530
1      5056
2       852
3       157
4        33
5         5
6         1
Name: NumOT, dtype: int64
NCAA
 0    1170
1      65
2      13
Name: NumOT, dtype: int64


In [6]:
reg_data_w =pd.read_csv('data/WRegularSeasonDetailedResults.csv')
reg_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


In [7]:
ncaa_data_w = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
ncaa_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12


In [8]:
#removing WLoc
print(reg_data_w['WLoc'].value_counts())
print(ncaa_data_w['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    39117
A    25410
N     6256
Name: WLoc, dtype: int64
N    492
H    228
A     40
Name: WLoc, dtype: int64


In [9]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_w['NumOT'].value_counts())
print("NCAA\n",ncaa_data_w['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    67642
1     2667
2      407
3       55
4       11
5        1
Name: NumOT, dtype: int64
NCAA
 0    734
1     23
2      3
Name: NumOT, dtype: int64


In [10]:
data = pd.concat([reg_data_m,ncaa_data_m,reg_data_w,ncaa_data_w])
print(data.shape)
print(data['Season'].value_counts())

(180425, 34)
2023    10976
2019    10833
2018    10744
2017    10735
2014    10718
2016    10708
2015    10698
2013    10593
2022    10539
2020    10499
2011    10460
2012    10433
2010    10427
2021     7540
2009     5313
2008     5227
2007     5107
2006     4821
2005     4739
2003     4680
2004     4635
Name: Season, dtype: int64


In [11]:
drop_cols = ['WLoc','NumOT']
reg_data = data.drop(columns = drop_cols)
cols = reg_data.columns.values
print(cols)
reg_data.head()

['Season' 'DayNum' 'WTeamID' 'WScore' 'LTeamID' 'LScore' 'WFGM' 'WFGA'
 'WFGM3' 'WFGA3' 'WFTM' 'WFTA' 'WOR' 'WDR' 'WAst' 'WTO' 'WStl' 'WBlk'
 'WPF' 'LFGM' 'LFGA' 'LFGM3' 'LFGA3' 'LFTM' 'LFTA' 'LOR' 'LDR' 'LAst'
 'LTO' 'LStl' 'LBlk' 'LPF']


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [12]:
# converting it to team A and team B from WTeam and LTeam
list_reg_data = []

for row in tqdm(reg_data.itertuples()):
    new_reg_data = dict()
    coin = random.random()
    new_reg_data['Season'] = row.Season
    new_reg_data['DayNum'] = row.DayNum

    if coin>0.5:
        new_reg_data['A'] = row.WTeamID
        new_reg_data['B'] = row.LTeamID
        try:
            new_reg_data['FGP2_A'] = ((row.WFGM - row.WFGM3)/ (row.WFGA - row.WFGA3)) * 100
        except:
            new_reg_data['FGP2_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        try:
            new_reg_data['AST_TO_A'] = row.WAst/row.WTO
        except:
            new_reg_data['AST_TO_A'] = row.WAst/(row.WTO+1)
        try:
            new_reg_data['STBL_PF_A'] = (row.WStl + row.WBlk)/row.WPF # steal + block / personal foul
        except:
            new_reg_data['STBL_PF_A'] = (row.WStl + row.WBlk)/(row.WPF+1)
        new_reg_data['PM_A'] = row.WScore - row.LScore # plus minus for team A
        new_reg_data['POSS_A'] = row.WFGA - row.WOR + row.WTO + (0.475*row.WFTA) # https://thepowerrank.com/cbb-analytics/ possession
        new_reg_data['REB_DIFF_A'] = (row.WOR/(row.WFGA-row.WFGM)) - (row.WDR/(row.LFGA-row.LFGM))
        try:
            new_reg_data['FGP2_B'] = ((row.LFGM - row.LFGM3)/ (row.LFGA - row.LFGA3)) * 100
        except:
            new_reg_data['FGP2_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        try:
            new_reg_data['AST_TO_B'] = row.LAst/row.LTO
        except:
            new_reg_data['AST_TO_B'] = row.LAst/(row.LTO+1)
        try:
            new_reg_data['STBL_PF_B'] = (row.LStl + row.LBlk)/row.LPF
        except:
            new_reg_data['STBL_PF_B'] = (row.LStl + row.LBlk)/(row.LPF+1)
        new_reg_data['PM_B'] = row.LScore - row.WScore # plus minus for team B
        new_reg_data['POSS_B'] = row.LFGA - row.LOR + row.LTO + (0.475*row.LFTA)
        new_reg_data['REB_DIFF_B'] = (row.LOR/(row.LFGA-row.LFGM)) - (row.LDR/(row.WFGA-row.WFGM))
        new_reg_data['Win'] = 1
    else:
        new_reg_data['A'] = row.LTeamID
        new_reg_data['B'] = row.WTeamID
        try:
            new_reg_data['FGP2_A'] = ((row.LFGM - row.LFGM3)/ (row.LFGA - row.LFGA3)) * 100
        except:
            new_reg_data['FGP2_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        try:
            new_reg_data['AST_TO_A'] = row.LAst/row.LTO
        except:
            new_reg_data['AST_TO_A'] = row.LAst/(row.LTO+1)
        try:
            new_reg_data['STBL_PF_A'] = (row.LStl + row.LBlk)/row.LPF
        except:
            new_reg_data['STBL_PF_A'] = (row.LStl + row.LBlk)/(row.LPF+1)
        new_reg_data['PM_A'] = row.LScore - row.WScore # plus minus for team A
        new_reg_data['POSS_A'] = row.LFGA - row.LOR + row.LTO + (0.475*row.LFTA)
        new_reg_data['REB_DIFF_A'] = (row.LOR/(row.LFGA-row.LFGM)) - (row.LDR/(row.WFGA-row.WFGM))
        try:
            new_reg_data['FGP2_B'] = ((row.WFGM - row.WFGM3)/ (row.WFGA - row.WFGA3)) * 100
        except:
            new_reg_data['FGP2_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        try:
            new_reg_data['AST_TO_B'] = row.WAst/row.WTO
        except:
            new_reg_data['AST_TO_B'] = row.WAst/(row.WTO+1)
        try:
            new_reg_data['STBL_PF_B'] = (row.WStl + row.WBlk)/row.WPF
        except:
            new_reg_data['STBL_PF_B'] = (row.WStl + row.WBlk)/(row.WPF+1)
        new_reg_data['PM_B'] = row.WScore - row.LScore # plus minus for team B
        new_reg_data['POSS_B'] = row.WFGA - row.WOR + row.WTO + (0.475*row.WFTA)
        new_reg_data['REB_DIFF_B'] = (row.WOR/(row.WFGA-row.WFGM)) - (row.WDR/(row.LFGA-row.LFGM))
        new_reg_data['Win'] = 0

    list_reg_data.append(new_reg_data)

new_reg_data = pd.DataFrame(list_reg_data)
new_reg_data


180425it [00:00, 217782.73it/s]


,Season,DayNum,A,B,FGP2_A,FGP3_A,FTP_A,AST_TO_A,STBL_PF_A,PM_A,POSS_A,REB_DIFF_A,FGP2_B,FGP3_B,FTP_B,AST_TO_B,STBL_PF_B,PM_B,POSS_B,REB_DIFF_B,Win
0,2003,10,1104,1328,54.545455,21.428571,61.111111,0.565217,0.363636,6,75.550,-0.322581,46.511628,20.000000,72.727273,0.444444,0.550000,-6,71.450,-0.387097,1
1,2003,10,1393,1272,41.860465,25.000000,45.000000,0.583333,0.875000,-7,68.500,-0.229328,42.857143,40.000000,52.631579,1.230769,0.444444,7,69.025,-0.234496,0
2,2003,11,1437,1266,40.425532,11.538462,60.869565,0.750000,0.304348,-12,64.925,-0.039216,40.000000,44.444444,58.620690,1.500000,0.280000,12,64.775,-0.009804,0
3,2003,11,1457,1296,44.444444,27.272727,53.333333,0.473684,0.304348,-6,58.125,-0.451613,51.724138,33.333333,54.838710,0.916667,0.888889,6,58.725,-0.312903,0
4,2003,11,1400,1208,51.063830,42.857143,84.615385,0.857143,0.400000,6,64.175,-0.030560,39.130435,37.500000,62.962963,1.200000,0.571429,-6,63.825,0.068761,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180420,2022,147,3163,3301,57.142857,23.809524,60.000000,1.428571,0.437500,4,81.500,-0.376471,58.139535,30.434783,84.210526,1.538462,0.687500,-4,82.025,-0.573529,1
180421,2022,147,3257,3276,46.511628,33.333333,77.777778,1.090909,1.117647,12,67.275,-0.484848,40.625000,21.428571,75.000000,0.428571,0.666667,-12,66.500,-0.393939,1
180422,2022,151,3163,3390,37.209302,35.714286,80.000000,0.736842,0.437500,5,73.500,-0.364341,44.186047,17.391304,61.538462,0.909091,0.875000,-5,72.175,-0.383075,1
180423,2022,151,3376,3257,52.500000,35.294118,70.588235,1.357143,1.363636,13,71.075,-0.400000,47.272727,12.500000,57.142857,0.333333,0.882353,-13,70.325,-0.294444,1


In [13]:
new_reg_data.iloc[:,4:].describe()

,FGP2_A,FGP3_A,FTP_A,AST_TO_A,STBL_PF_A,PM_A,POSS_A,REB_DIFF_A,FGP2_B,FGP3_B,FTP_B,AST_TO_B,STBL_PF_B,PM_B,POSS_B,REB_DIFF_B,Win
count,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000
mean,46.812609,33.017732,69.419273,1.024671,0.777734,-0.039490,69.416604,-0.400910,46.845399,33.047129,69.410606,1.027017,0.777865,0.039490,69.420209,-0.400999,0.498456
std,9.815755,12.038267,13.091676,0.610840,1.313548,16.345619,6.792261,0.158261,9.826228,12.019730,13.108246,0.604855,1.318765,16.345619,6.791424,0.157979,0.499999
min,4.166667,0.000000,0.000000,0.000000,0.000000,-108.000000,41.425000,-1.340000,6.060606,0.000000,0.000000,0.000000,0.000000,-93.000000,41.800000,-1.481707,0.000000
25%,40.000000,25.000000,61.538462,0.636364,0.400000,-11.000000,64.800000,-0.506250,40.000000,25.000000,61.538462,0.636364,0.400000,-11.000000,64.800000,-0.505981,0.000000
50%,46.666667,33.333333,70.000000,0.900000,0.571429,-1.000000,69.025000,-0.400000,46.666667,33.333333,70.000000,0.909091,0.571429,1.000000,69.025000,-0.400000,0.000000
75%,53.333333,40.909091,78.260870,1.266667,0.785714,11.000000,73.550000,-0.293651,53.333333,40.909091,78.260870,1.266667,0.785714,11.000000,73.550000,-0.294118,1.000000
max,96.428571,100.000000,100.000000,29.000000,25.000000,93.000000,128.350000,0.512488,93.333333,100.000000,100.000000,25.000000,26.000000,108.000000,126.850000,0.336991,1.000000


In [14]:
new_reg_data['Win'].value_counts()

0    90491
1    89934
Name: Win, dtype: int64

In [15]:
new_reg_data.corr()

,Season,DayNum,A,B,FGP2_A,FGP3_A,FTP_A,AST_TO_A,STBL_PF_A,PM_A,POSS_A,REB_DIFF_A,FGP2_B,FGP3_B,FTP_B,AST_TO_B,STBL_PF_B,PM_B,POSS_B,REB_DIFF_B,Win
Season,1.000000,-0.020767,0.288956,0.289189,-0.003610,-0.051570,0.056652,0.047937,0.149516,-0.000176,0.130023,-0.150867,-0.006830,-0.054770,0.054543,0.049996,0.149227,0.000176,0.130759,-0.151138,-0.000523
DayNum,-0.020767,1.000000,-0.011243,-0.011263,0.016297,0.022372,0.064749,0.069054,-0.041487,0.003448,-0.151446,-0.051711,0.014617,0.021281,0.062563,0.064816,-0.042240,-0.003448,-0.151516,-0.050205,0.001221
A,0.288956,-0.011243,1.000000,0.988725,-0.253428,-0.130750,-0.005853,-0.150410,-0.013938,0.002831,0.185614,0.172197,-0.260769,-0.128388,-0.007429,-0.154288,-0.014871,-0.002831,0.186112,0.171837,0.003793
B,0.289189,-0.011263,0.988725,1.000000,-0.254956,-0.131025,-0.006681,-0.151648,-0.013491,-0.000100,0.186323,0.170928,-0.258735,-0.128253,-0.007320,-0.153353,-0.014057,0.000100,0.186766,0.172330,0.000946
FGP2_A,-0.003610,0.016297,-0.253428,-0.254956,1.000000,0.109286,0.061781,0.303525,0.048655,0.473516,0.014527,-0.040466,-0.015519,0.006415,0.008561,-0.019471,-0.045884,-0.473516,0.020703,-0.170765,0.377101
FGP3_A,-0.051570,0.022372,-0.130750,-0.131025,0.109286,1.000000,0.056192,0.268458,-0.000177,0.354911,-0.042653,-0.012459,0.002983,0.009815,0.013964,0.031779,-0.021093,-0.354911,-0.019816,-0.083492,0.303237
FTP_A,0.056652,0.064749,-0.005853,-0.006681,0.061781,0.056192,1.000000,0.057604,0.006198,0.155741,0.011866,-0.072301,0.008103,0.010413,0.038727,0.021618,-0.012312,-0.155741,0.009651,0.110072,0.136434
AST_TO_A,0.047937,0.069054,-0.150410,-0.151648,0.303525,0.268458,0.057604,1.000000,0.052332,0.423474,-0.111077,-0.081405,-0.016984,0.030640,0.017484,0.046633,-0.074019,-0.423474,-0.101645,-0.102512,0.319682
STBL_PF_A,0.149516,-0.041487,-0.013938,-0.013491,0.048655,-0.000177,0.006198,0.052332,1.000000,0.104285,0.071461,0.006384,-0.045930,-0.023411,-0.013161,-0.075709,0.813310,-0.104285,0.075083,-0.025022,0.090196
PM_A,-0.000176,0.003448,0.002831,-0.000100,0.473516,0.354911,0.155741,0.423474,0.104285,1.000000,0.001859,0.063477,-0.472858,-0.355072,-0.156154,-0.424801,-0.104404,-1.000000,-0.002465,-0.062332,0.790040


### Few heavy correlation
- FGP_A - FGP3_A (0.53)
- FGP_A - AST_A (0.55)
- FGP_A - DR_B (negative)
- FGP3_A - AST_A
- TO_A - STL_B (quite high, 0.71)
- STL_A - TO_B (quite high, 0.71)
same for B

In [16]:
def get_avg_team(data, team_id, dn):
    data = data[(data['A'] == team_id)|(data['B'] == team_id)]
    data = data[data['DayNum']<dn]
    final_data = {'FGP2_scored':[],'FGP3_scored':[],'FTP_scored':[],
                  'AST_TO_scored':[],'STBL_PF_scored':[],
                  'POSS_scored' : [], 'REB_DIFF_scored':[],
                  'FGP2_allowed':[],'FGP3_allowed':[],'FTP_allowed':[],
                  'AST_TO_allowed':[],'STBL_PF_allowed':[],
                  'POSS_allowed':[], 'REB_DIFF_allowed':[],'PM':[]}
    for row in data.itertuples():
        if row.A == team_id:
            final_data['FGP2_scored'].append(row.FGP2_A)
            final_data['FGP3_scored'].append(row.FGP3_A)
            final_data['FTP_scored'].append(row.FTP_A)
            final_data['AST_TO_scored'].append(row.AST_TO_A)
            final_data['STBL_PF_scored'].append(row.STBL_PF_A)
            final_data['POSS_scored'].append(row.POSS_A)
            final_data['REB_DIFF_scored'].append(row.REB_DIFF_A)
            final_data['FGP2_allowed'].append(row.FGP2_B)
            final_data['FGP3_allowed'].append(row.FGP3_B)
            final_data['FTP_allowed'].append(row.FTP_B)
            final_data['AST_TO_allowed'].append(row.AST_TO_B)
            final_data['STBL_PF_allowed'].append(row.STBL_PF_B)
            final_data['POSS_allowed'].append(row.POSS_B)
            final_data['REB_DIFF_allowed'].append(row.REB_DIFF_B)
            final_data['PM'].append(row.PM_A)
        else:
            final_data['FGP2_scored'].append(row.FGP2_B)
            final_data['FGP3_scored'].append(row.FGP3_B)
            final_data['FTP_scored'].append(row.FTP_B)
            final_data['AST_TO_scored'].append(row.AST_TO_B)
            final_data['STBL_PF_scored'].append(row.STBL_PF_B)
            final_data['POSS_scored'].append(row.POSS_B)
            final_data['REB_DIFF_scored'].append(row.REB_DIFF_B)
            final_data['FGP2_allowed'].append(row.FGP2_A)
            final_data['FGP3_allowed'].append(row.FGP3_A)
            final_data['FTP_allowed'].append(row.FTP_A)
            final_data['AST_TO_allowed'].append(row.AST_TO_A)
            final_data['STBL_PF_allowed'].append(row.STBL_PF_A)
            final_data['POSS_allowed'].append(row.POSS_A)
            final_data['REB_DIFF_allowed'].append(row.REB_DIFF_A)
            final_data['PM'].append(row.PM_B)

    try:
        return [sum(v)/len(v) for v in final_data.values()]
    except:
        return [np.nan]*len(final_data)


def get_avg_yearly(year):
    year_data = new_reg_data[new_reg_data['Season'] == year]
    list_data = []

    for row in tqdm(year_data.itertuples()):
        *stats_A, = get_avg_team(year_data, row.A, row.DayNum)
        *stats_B, = get_avg_team(year_data, row.B, row.DayNum)
        list_data.append([row.Season, row.DayNum, row.A, row.B, *stats_A, *stats_B, row.Win])
    
    data = pd.DataFrame(list_data, columns = ['Season','DayNum','A','B','FGP2_SCORED_A','FGP3_SCORED_A','FTP_SCORED_A',
                                               'AST_TO_SCORED_A','STBL_PF_SCORED_A','POSS_SCORED_A','REB_DIFF_SCORED_A',
                                               'FGP2_ALLOWED_A', 'FGP3_ALLOWED_A','FTP_ALLOWED_A','AST_TO_ALLOWED_A',
                                               'STBL_PF_ALLOWED_A','POSS_ALLOWED_A','REB_DIFF_ALLOWED_A','PM_A','FGP2_SCORED_B',
                                               'FGP3_SCORED_B','FTP_SCORED_B','AST_TO_SCORED_B','STBL_PF_SCORED_B',
                                               'POSS_SCORED_B','REB_DIFF_SCORED_B','FGP2_ALLOWED_B', 'FGP3_ALLOWED_B','FTP_ALLOWED_B',
                                               'AST_TO_ALLOWED_B', 'STBL_PF_ALLOWED_B','POSS_ALLOWED_B','REB_DIFF_ALLOWED_B','PM_B','Win']).dropna().reset_index(drop=True)

    return data

# to get an understanding of margin difference between the two teams
# Offense_A, Defense_B
# Offense_B, Defense_A
def margin_diff(data):
    final_data = {'Season':[],'DayNum':[],'A':[],'B':[],'PM_A':[],'PM_B':[],'margin_FGP2_A':[],'margin_FGP3_A':[],'margin_FTP_A':[],'margin_AST_TO_A':[],
                  'margin_STBL_PF_A':[],'margin_REB_DIFF_A':[], 'margin_POSS_A':[],'margin_FGP2_B':[],
                  'margin_FGP3_B':[],'margin_FTP_B':[],'margin_AST_TO_B':[],'margin_STBL_PF_B':[],
                  'margin_REB_DIFF_B':[],'margin_POSS_B':[],'Win':[]}
        
    for row in data.itertuples():
        final_data['Season'].append(row.Season)
        final_data['DayNum'].append(row.DayNum)
        final_data['A'].append(row.A)
        final_data['B'].append(row.B)
        final_data['PM_A'].append(row.PM_A)
        final_data['PM_B'].append(row.PM_B)
        final_data['margin_FGP2_A'].append(row.FGP2_SCORED_A - row.FGP2_ALLOWED_B)
        final_data['margin_FGP3_A'].append(row.FGP3_SCORED_A - row.FGP3_ALLOWED_B)
        final_data['margin_FTP_A'].append(row.FTP_SCORED_A - row.FTP_ALLOWED_B)
        final_data['margin_AST_TO_A'].append(row.AST_TO_SCORED_A - row.AST_TO_ALLOWED_B)
        final_data['margin_STBL_PF_A'].append(row.STBL_PF_SCORED_A - row.STBL_PF_ALLOWED_B)
        final_data['margin_REB_DIFF_A'].append(row.REB_DIFF_SCORED_A - row.REB_DIFF_ALLOWED_B)
        final_data['margin_POSS_A'].append(row.POSS_SCORED_A - row.POSS_ALLOWED_B)
        final_data['margin_FGP2_B'].append(row.FGP2_SCORED_B - row.FGP2_ALLOWED_A)
        final_data['margin_FGP3_B'].append(row.FGP3_SCORED_B - row.FGP3_ALLOWED_A)
        final_data['margin_FTP_B'].append(row.FTP_SCORED_B - row.FTP_ALLOWED_A)
        final_data['margin_AST_TO_B'].append(row.AST_TO_SCORED_B - row.AST_TO_ALLOWED_A)
        final_data['margin_STBL_PF_B'].append(row.STBL_PF_SCORED_B - row.STBL_PF_ALLOWED_A)
        final_data['margin_REB_DIFF_B'].append(row.REB_DIFF_SCORED_B - row.REB_DIFF_ALLOWED_A)
        final_data['margin_POSS_B'].append(row.POSS_SCORED_B - row.POSS_ALLOWED_A)
        final_data['Win'].append(row.Win)

    final_data = pd.DataFrame(final_data)
    return final_data

def get_data_reg():
    year = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    data = []
    for y in tqdm(year):
        data.append(get_avg_yearly(y))

    final_data = pd.concat(data)
    final_data = margin_diff(final_data)
    return final_data


In [17]:
data_03 = get_avg_yearly(2003)


4680it [00:11, 413.40it/s]


In [18]:
data_03.head()

,Season,DayNum,A,B,FGP2_SCORED_A,FGP3_SCORED_A,FTP_SCORED_A,AST_TO_SCORED_A,STBL_PF_SCORED_A,POSS_SCORED_A,REB_DIFF_SCORED_A,FGP2_ALLOWED_A,FGP3_ALLOWED_A,FTP_ALLOWED_A,AST_TO_ALLOWED_A,STBL_PF_ALLOWED_A,POSS_ALLOWED_A,REB_DIFF_ALLOWED_A,PM_A,FGP2_SCORED_B,FGP3_SCORED_B,FTP_SCORED_B,AST_TO_SCORED_B,STBL_PF_SCORED_B,POSS_SCORED_B,REB_DIFF_SCORED_B,FGP2_ALLOWED_B,FGP3_ALLOWED_B,FTP_ALLOWED_B,AST_TO_ALLOWED_B,STBL_PF_ALLOWED_B,POSS_ALLOWED_B,REB_DIFF_ALLOWED_B,PM_B,Win
0,2003,12,1186,1457,48.571429,27.272727,70.588235,0.421053,0.280000,67.075,-0.478908,44.444444,50.000000,85.185185,1.333333,0.666667,66.825,-0.535980,-26.0,44.444444,27.272727,53.333333,0.473684,0.304348,58.125,-0.451613,51.724138,33.333333,54.838710,0.916667,0.888889,58.725,-0.312903,-6.0,1
1,2003,12,1296,1458,51.724138,33.333333,54.838710,0.916667,0.888889,58.725,-0.312903,44.444444,27.272727,53.333333,0.473684,0.304348,58.125,-0.451613,6.0,44.444444,50.000000,85.185185,1.333333,0.666667,66.825,-0.535980,48.571429,27.272727,70.588235,0.421053,0.280000,67.075,-0.478908,26.0,0
2,2003,14,1135,1125,48.648649,31.578947,64.705882,0.944444,0.923077,68.075,-0.087500,44.680851,26.315789,69.230769,0.882353,0.882353,68.175,-0.193750,-1.0,25.000000,33.333333,50.000000,0.705882,0.588235,70.800,-0.534362,45.454545,43.478261,69.565217,1.384615,2.538462,71.925,-0.502805,-28.0,0
3,2003,14,1156,1236,52.941176,33.333333,44.444444,0.520000,0.555556,76.825,-0.402381,48.936170,45.454545,55.555556,0.529412,0.478261,74.550,-0.485714,-5.0,57.692308,26.666667,71.428571,0.366667,0.500000,75.300,-0.247159,44.680851,25.000000,82.051282,0.823529,0.480000,77.525,-0.411932,-18.0,1
4,2003,14,1194,1161,48.936170,45.454545,55.555556,0.529412,0.478261,74.550,-0.485714,52.941176,33.333333,44.444444,0.520000,0.555556,76.825,-0.402381,5.0,44.680851,25.000000,82.051282,0.823529,0.480000,77.525,-0.411932,57.692308,26.666667,71.428571,0.366667,0.500000,75.300,-0.247159,18.0,0


In [19]:
data_03_margin = margin_diff(data_03)

In [20]:
data_03_margin

,Season,DayNum,A,B,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
0,2003,12,1186,1457,-26.000000,-6.000000,-3.152709,-6.060606,15.749526,-0.495614,-0.608889,-0.166005,8.350000,0.000000,-22.727273,-31.851852,-0.859649,-0.362319,0.084367,-8.700000,1
1,2003,12,1296,1458,6.000000,26.000000,3.152709,6.060606,-15.749526,0.495614,0.608889,0.166005,-8.350000,0.000000,22.727273,31.851852,0.859649,0.362319,-0.084367,8.700000,0
2,2003,14,1135,1125,-1.000000,-28.000000,3.194103,-11.899314,-4.859335,-0.440171,-1.615385,0.415305,-3.850000,-19.680851,7.017544,-19.230769,-0.176471,-0.294118,-0.340612,2.625000,0
3,2003,14,1156,1236,-5.000000,-18.000000,8.260325,8.333333,-37.606838,-0.303529,0.075556,0.009551,-0.700000,8.756137,-18.787879,15.873016,-0.162745,0.021739,0.238555,0.750000,1
4,2003,14,1194,1161,5.000000,18.000000,-8.756137,18.787879,-15.873016,0.162745,-0.021739,-0.238555,-0.750000,-8.260325,-8.333333,37.606838,0.303529,-0.075556,-0.009551,0.700000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4472,2003,146,1328,1393,11.121212,10.000000,3.491727,9.549893,3.188075,0.125275,0.075790,-0.103482,-7.814157,7.684522,2.151693,3.205801,0.401816,0.544782,0.136700,7.471236,0
4473,2003,146,1400,1277,10.322581,6.852941,4.218556,2.519487,1.009623,0.288299,0.022799,0.141285,6.022723,4.743501,2.547989,8.203791,0.069595,-0.015639,-0.159746,-5.349407,1
4474,2003,152,1242,1266,14.441176,10.312500,7.885769,2.281373,-4.211293,0.106029,0.510847,-0.013997,6.556939,9.387248,4.917044,8.922371,0.565818,-0.050506,0.133548,-5.333869,1
4475,2003,152,1393,1400,10.181818,10.281250,8.414924,-1.136568,4.869187,0.288810,0.491693,0.015461,1.951042,4.522181,5.196445,5.675823,0.052845,-0.018130,0.012254,-1.452699,1


In [21]:
data_03_margin.corr()

,Season,DayNum,A,B,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
Season,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DayNum,NaN,1.000000,0.024941,-0.002312,0.040326,0.038717,0.050261,0.012818,-0.012603,0.039757,0.033783,0.003270,0.014688,0.030193,0.017733,0.030013,0.035523,0.025446,0.016302,-0.014353,0.009215
A,NaN,0.024941,1.000000,0.006648,0.037954,0.025286,0.031555,-0.012258,0.033838,0.046327,0.032719,0.000376,-0.007272,0.033870,-0.006837,0.000539,0.008077,0.031069,-0.010176,0.005490,0.021869
B,NaN,-0.002312,0.006648,1.000000,0.031226,0.033715,0.019601,-0.013568,0.034233,0.010909,0.045762,-0.004670,-0.000375,0.041295,0.010905,0.022515,0.055166,0.027438,-0.007250,-0.009991,-0.011739
PM_A,NaN,0.040326,0.037954,0.031226,1.000000,0.249315,0.548379,0.345950,0.170218,0.517800,0.482134,0.124493,0.047540,0.489690,0.290284,0.136999,0.471213,0.470609,0.124240,-0.015643,0.244272
PM_B,NaN,0.038717,0.025286,0.033715,0.249315,1.000000,0.493809,0.296758,0.134261,0.492578,0.474505,0.113060,0.001681,0.549617,0.351765,0.178916,0.519207,0.487694,0.133052,0.028102,-0.238492
margin_FGP2_A,NaN,0.050261,0.031555,0.019601,0.548379,0.493809,1.000000,0.204433,0.116583,0.486276,0.315249,0.099071,0.034539,0.316642,0.166728,0.010443,0.301705,0.476629,0.196848,-0.014607,0.023676
margin_FGP3_A,NaN,0.012818,-0.012258,-0.013568,0.345950,0.296758,0.204433,1.000000,0.135614,0.343784,0.127589,-0.035430,-0.008388,0.185456,0.056410,0.048460,0.104761,0.133708,0.131768,0.013181,0.053126
margin_FTP_A,NaN,-0.012603,0.033838,0.034233,0.170218,0.134261,0.116583,0.135614,1.000000,0.098154,0.031244,-0.051341,0.001388,0.014978,0.029109,-0.033905,0.065499,0.150640,-0.113798,0.020083,0.046958
margin_AST_TO_A,NaN,0.039757,0.046327,0.010909,0.517800,0.492578,0.486276,0.343784,0.098154,1.000000,0.375431,0.025142,-0.082627,0.279993,0.139434,0.110542,0.207755,0.442638,0.126265,0.104290,0.026119


In [22]:
final_data = get_data_reg()

4680it [00:11, 409.33it/s]00<?, ?it/s]
4635it [00:11, 410.15it/s]11<03:49, 11.45s/it]
4739it [00:11, 408.39it/s]22<03:36, 11.38s/it]
4821it [00:11, 414.87it/s]34<03:26, 11.49s/it]
5107it [00:12, 412.73it/s]46<03:16, 11.55s/it]
5227it [00:12, 418.28it/s]58<03:09, 11.86s/it]
5313it [00:12, 412.16it/s]10<03:01, 12.08s/it]
10427it [00:25, 407.34it/s]3<02:52, 12.35s/it]
10460it [00:25, 412.98it/s]9<03:35, 16.59s/it]
10433it [00:24, 418.48it/s]4<03:52, 19.34s/it]
10593it [00:25, 421.14it/s]39<03:51, 21.08s/it]
10718it [00:25, 418.10it/s]05<03:43, 22.34s/it]
10698it [00:25, 422.75it/s]30<03:30, 23.36s/it]
10708it [00:25, 422.27it/s]56<03:11, 23.96s/it]
10735it [00:25, 420.23it/s]21<02:50, 24.40s/it]
10744it [00:25, 419.91it/s]47<02:28, 24.76s/it]
10833it [00:25, 420.00it/s]12<02:05, 25.03s/it]
10499it [00:24, 422.92it/s]38<01:41, 25.27s/it]
7540it [00:17, 426.65it/s]:03<01:15, 25.15s/it]
10539it [00:25, 419.87it/s]21<00:45, 22.92s/it]
10976it [00:26, 421.20it/s]46<00:23, 23.59s/it]
100%|█████

In [23]:
final_data.to_csv('computed_data/reg_avg_data.csv',index=False)

In [24]:
data_03.to_csv('computed_data/data_03.csv',index=False)

In [25]:
final_data = pd.read_csv('computed_data/reg_avg_data.csv')

In [26]:
final_data.shape

(173017, 21)

In [27]:
data_03 = pd.read_csv('computed_data/data_03.csv')
data_03[data_03['A']==1457]

,Season,DayNum,A,B,FGP2_SCORED_A,FGP3_SCORED_A,FTP_SCORED_A,AST_TO_SCORED_A,STBL_PF_SCORED_A,POSS_SCORED_A,REB_DIFF_SCORED_A,FGP2_ALLOWED_A,FGP3_ALLOWED_A,FTP_ALLOWED_A,AST_TO_ALLOWED_A,STBL_PF_ALLOWED_A,POSS_ALLOWED_A,REB_DIFF_ALLOWED_A,PM_A,FGP2_SCORED_B,FGP3_SCORED_B,FTP_SCORED_B,AST_TO_SCORED_B,STBL_PF_SCORED_B,POSS_SCORED_B,REB_DIFF_SCORED_B,FGP2_ALLOWED_B,FGP3_ALLOWED_B,FTP_ALLOWED_B,AST_TO_ALLOWED_B,STBL_PF_ALLOWED_B,POSS_ALLOWED_B,REB_DIFF_ALLOWED_B,PM_B,Win
359,2003,29,1457,1428,44.567558,31.209307,66.899314,0.774671,0.613879,67.693750,-0.550016,51.323892,36.309524,63.908090,0.906768,0.534197,67.556250,-0.464005,-3.750000,52.894265,37.211400,54.665242,1.084295,0.474887,66.525000,-0.388832,44.291346,33.409091,68.385969,0.778571,0.522234,61.250000,-0.284702,1.750000,0
841,2003,45,1457,1155,43.087430,26.201056,64.645413,0.838910,0.690564,67.496429,-0.458919,44.959251,38.601006,60.405736,0.914932,0.565892,67.150000,-0.377872,-6.571429,53.440892,29.839487,60.772163,1.117316,0.673464,67.690000,-0.125012,45.184914,33.050697,59.672439,0.746314,0.454391,69.115000,-0.199539,12.800000,0
1488,2003,61,1457,1189,44.738624,27.968190,62.453752,0.801272,0.572180,67.987500,-0.439005,44.086860,41.347977,62.836956,0.951246,0.547314,67.632500,-0.360173,-6.100000,45.954058,30.566893,62.384837,0.650705,0.378311,67.132143,-0.304964,53.612050,39.359946,78.272135,1.218398,0.861970,64.042857,-0.260787,-13.857143,1
1966,2003,73,1457,1251,46.687736,31.738351,62.801660,0.785125,0.601403,66.992308,-0.420195,43.444382,40.397224,63.297101,0.926416,0.545935,66.751923,-0.369805,-3.923077,46.485488,32.267843,73.051590,0.736509,0.599205,72.573214,-0.334999,47.434707,36.634311,65.587298,0.834436,0.603503,73.583929,-0.339898,-3.857143,1
2112,2003,75,1457,1189,47.000314,32.532551,62.232878,0.762658,0.615589,67.258929,-0.404611,43.242997,40.306739,63.686594,0.901060,0.536160,67.169643,-0.369846,-2.928571,44.068829,32.719074,59.995823,0.682510,0.412985,66.443182,-0.330964,53.742497,39.447775,79.564694,1.264934,0.834938,64.177273,-0.328073,-15.272727,1
2521,2003,86,1457,1149,46.878951,32.989008,63.270733,0.783339,0.602950,66.957813,-0.407336,43.608792,38.928156,63.486187,0.898919,0.515185,66.892187,-0.380929,-1.937500,50.580849,28.028303,72.815731,1.062583,0.751939,67.681250,-0.416866,46.369570,37.825243,68.259275,0.840136,0.769989,68.603125,-0.372873,-0.500000,1
2583,2003,88,1457,1219,46.889532,32.858433,63.078336,0.782509,0.653043,67.225000,-0.396447,44.157756,37.814735,64.793722,0.897512,0.525321,67.152941,-0.378130,-1.647059,50.590840,31.988045,60.729726,0.757800,0.515082,72.219118,-0.385234,55.258455,36.082571,67.233023,1.032100,0.678777,70.755882,-0.384709,-4.705882,1
2837,2003,93,1457,1347,47.895669,33.079748,62.484037,0.805009,0.679263,67.655556,-0.395538,44.008070,37.797250,63.971848,0.894434,0.547988,67.513889,-0.385151,-0.166667,37.548879,31.785828,66.772506,0.643820,0.461023,70.401250,-0.438189,52.022477,34.978991,68.088353,1.234385,0.603138,70.606250,-0.355372,-14.250000,1
2913,2003,96,1457,1157,47.590911,34.189586,62.600977,0.828429,0.675549,67.993421,-0.395376,43.581195,37.061054,64.723902,0.878318,0.541703,67.851316,-0.381996,0.947368,47.182078,36.033823,72.423833,0.793429,0.385474,67.245833,-0.352832,49.567490,33.330404,69.919475,0.964989,0.790317,67.608333,-0.283830,-3.555556,0
3081,2003,98,1457,1128,47.572477,33.567063,62.619076,0.824508,0.660522,68.135000,-0.384301,43.366421,37.253456,65.453224,0.871069,0.530618,67.797500,-0.375329,0.600000,51.611295,34.691730,67.791016,0.970033,0.685359,65.012500,-0.329112,47.568911,37.425469,71.421875,0.832228,0.521453,64.760000,-0.356193,2.300000,1
